In this notebook, I will write a function that tells whether a wrestler has seven wins in the current tournament.

First, we'll make a pandas dataframe.

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

#Add the path to the folder that contains the data. Makes it easier for each of us to use the notebook
path = "/Users/Kyla/Desktop/ErdosDataScienceWork/Project/data/"
#path = "/Users/es/Projects/erdo/sumo/Erdos-Institute-Fall-2024-Project-Sumo/data/"

df1983 = pd.read_csv(path+str(1983)+'.csv')
df1984 = pd.read_csv(path+str(1984)+'.csv')

df = df1983
for year in range(1984,2020):
    year = str(year)
    dfyear = pd.read_csv(path+year+'.csv')
    df = pd.concat([df, dfyear], axis=0, ignore_index=True)

In [2]:
df

,index,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214783,4985,2019.09,15,12210,S1e,Mitakeumi,12-3,1,yorikiri,12055,K1w,Endo,8-7,0
214784,4986,2019.09,15,6468,O1w,Goeido,10-5,1,yorikiri,6599,O2e,Tochinoshin,6-9,0
214785,4987,2019.09,15,6599,O2e,Tochinoshin,6-9,0,yorikiri,6468,O1w,Goeido,10-5,1
214786,4988,2019.09,16,12191,S1w,Takakeisho,(12-3),0,yorikiri,12210,S1e,Mitakeumi,(12-3),1


Now let's extract the number of wins.

In [ ]:
# Define the function to calculate wins
def calculate_wins(result):
    # Initialize wins with None
    wins = None
    
    # Ensure result has at least three elements for safe indexing
    if len(result) < 3:
        return wins

    # Check if the result starts with '(' and matches other conditions
    if result[0] == '(':
        if result[2] != '-':
            wins = result[1] + result[2]  # Concatenate first and second characters if second character is '0'
        elif result[1].isdigit() and int(result[1]) < 10:
            wins = result[1]  # If result[1] is a single digit, assign it to wins as an integer

    # Check alternative conditions if not wrapped in parentheses
    elif result[1] != '-':
        wins = result[0] + result[1]
    elif result[0].isdigit() and int(result[0]) < 10:
        wins = int(result[0])

    return wins

# Apply the function to each row in 'rikishi1_result' and create the new column
df['rikishi1_wins_in_tournament'] = df['rikishi1_result'].apply(calculate_wins)
df['rikishi2_wins_in_tournament'] = df['rikishi2_result'].apply(calculate_wins)


In [ ]:
df

This seems to work! Thank you chatgpt! Note that entries in the new column are integers, not strings.

How many wins do wrestlers have on a given day of the current tournament?

In [ ]:
day_14_df = df[df['day'] == 14] #Just show the rows for matches on Day 14 of tournaments.
day_14_df

Now let's add a column. This column will have None in every entry. We'll change it later.

In [ ]:
def rikishi1_penultimate_day_7_wins(result):
    answer = None
    return answer

df['rikishi1_penultimate_day_7_wins'] = df['rikishi1_wins_in_tournament'].apply(rikishi1_penultimate_day_7_wins)

df['rikishi2_penultimate_day_7_wins'] = None

In [ ]:
df

If it's day 14 of a tournament, replace None with either True or False depending on whether rikishi1 has 7 wins or not.

In [ ]:
for i in range(df.shape[0]):
    if df['day'][i] == 14:
        if df['rikishi1_wins_in_tournament'][i] == 7:
            df.loc[i,'rikishi1_penultimate_day_7_wins'] = True
        if df['rikishi1_wins_in_tournament'][i] != 7:
            df.loc[i,'rikishi1_penultimate_day_7_wins'] = False

        if df['rikishi2_wins_in_tournament'][i] == 7:
            df.loc[i,'rikishi2_penultimate_day_7_wins'] = True
        if df['rikishi2_wins_in_tournament'][i] != 7:
            df.loc[i,'rikishi2_penultimate_day_7_wins'] = False

In [ ]:
df

Quick sanity check: let's look at all the day 14 rows. If it's the penultimate day, we should always get True or False, otherwise we should get None.

In [ ]:
day_14_df = df[df['day'] == 14]
day_14_df

Hey, this looks like it's working! Now we should rearrange the columns.


In [ ]:
features = df.columns
features

In [ ]:
new_order = ['index', 'basho', 'day', 'rikishi1_id', 'rikishi1_rank',
       'rikishi1_shikona', 'rikishi1_result', 'rikishi1_win', 'rikishi1_wins_in_tournament', 
       'rikishi1_penultimate_day_7_wins', 'kimarite',
       'rikishi2_id', 'rikishi2_rank', 'rikishi2_shikona', 'rikishi2_result',
       'rikishi2_win', 'rikishi2_wins_in_tournament', 'rikishi2_penultimate_day_7_wins']
df = df[new_order]

In [ ]:
df

The next cell contains a functions which takes in a data frame and outputs a data frame with a column added for "rikishi1_penultimate_day_7_wins."

In [4]:
def add_col_for_penultimate_day_7_wins(datfram):

    # Define the function to calculate wins
    def calculate_wins(result):
        # Initialize wins with None
        wins = None
        
        # Ensure result has at least three elements for safe indexing
        if len(result) < 3:
            return wins

        # Check if the result starts with '(' and matches other conditions
        if result[0] == '(':
            if result[2] != '-':
                wins = result[1] + result[2]  # Concatenate first and second characters if second character is '0'
            elif result[1].isdigit() and int(result[1]) < 10:
                wins = result[1]  # If result[1] is a single digit, assign it to wins as an integer

        # Check alternative conditions if not wrapped in parentheses
        elif result[1] != '-':
            wins = result[0] + result[1]
        elif result[0].isdigit() and int(result[0]) < 10:
            wins = int(result[0])

        return wins

    # Apply the function to each row in 'rikishi1_result' and create the new column
    df['rikishi1_wins_in_tournament'] = df['rikishi1_result'].apply(calculate_wins)
    df['rikishi2_wins_in_tournament'] = df['rikishi2_result'].apply(calculate_wins)


    datfram['rikishi1_penultimate_day_7_wins'] = None
    datfram['rikishi2_penultimate_day_7_wins'] = None

    for i in range(datfram.shape[0]):
        if datfram['day'][i] == 14:
            if datfram['rikishi1_wins_in_tournament'][i] == 7:
                datfram.loc[i,'rikishi1_penultimate_day_7_wins'] = True
            if datfram['rikishi1_wins_in_tournament'][i] != 7:
                datfram.loc[i,'rikishi1_penultimate_day_7_wins'] = False

            if datfram['rikishi2_wins_in_tournament'][i] == 7:
                datfram.loc[i,'rikishi2_penultimate_day_7_wins'] = True
            if datfram['rikishi2_wins_in_tournament'][i] != 7:
                datfram.loc[i,'rikishi2_penultimate_day_7_wins'] = False
    
    return datfram

Let's test this out.

In [5]:
add_col_for_penultimate_day_7_wins(df)

,index,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win,rikishi1_wins_in_tournament,rikishi2_wins_in_tournament,rikishi1_penultimate_day_7_wins,rikishi2_penultimate_day_7_wins
0,0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1,0,1,None,None
1,1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0,1,0,None,None
2,2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0,1,0,None,None
3,3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1,0,1,None,None
4,4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1,0,1,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214783,4985,2019.09,15,12210,S1e,Mitakeumi,12-3,1,yorikiri,12055,K1w,Endo,8-7,0,12,8,None,None
214784,4986,2019.09,15,6468,O1w,Goeido,10-5,1,yorikiri,6599,O2e,Tochinoshin,6-9,0,10,6,None,None
214785,4987,2019.09,15,6599,O2e,Tochinoshin,6-9,0,yorikiri,6468,O1w,Goeido,10-5,1,6,10,None,None
214786,4988,2019.09,16,12191,S1w,Takakeisho,(12-3),0,yorikiri,12210,S1e,Mitakeumi,(12-3),1,12,12,None,None
